In [1]:
import torch
import sys

print("Python version:", sys.version)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("GPU count:", torch.cuda.device_count())
    
    # Quick tensor test
    x = torch.randn(1000, 1000).cuda()
    y = torch.randn(1000, 1000).cuda()
    z = torch.matmul(x, y)
    print("GPU computation successful!")
else:
    print("No GPU detected!")

Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:40:08) [MSC v.1938 64 bit (AMD64)]
PyTorch version: 2.5.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 3070
GPU count: 1
GPU computation successful!


In [3]:
import sys
import os
import json
import yaml
from datetime import datetime, timezone
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
# Milyen modellel dolgozunk? és hogyan várja az adatokat?
# Nem mindehol van volume oszlop mert ez is elég szarul volt specifikálva a feladatkiírásban. Tehát lehet még azt is ki kell dobni ahol van
# maradjunk a data with labelsnél majd kicsit átírjuk az adatfeldolgozást ha jól működik ez a pole hozzáadás és kell is


In [5]:
DATA_DIR = Path("C:/msc_2/DL/vitmma19-pw-bullflag/data")
PROCESSED_FILE = DATA_DIR / "processed_data.npz"


def load_processed_dataset(npz_file: Path) -> dict[str, np.ndarray]:
    if not npz_file.exists():
        raise FileNotFoundError(f"Missing file: {npz_file}")
    with np.load(npz_file) as data:
        return {key: data[key] for key in data.files}


dataset = load_processed_dataset(PROCESSED_FILE)
for key, array in dataset.items():
    print(f"{key}: shape={array.shape}, dtype={array.dtype}")

train_X: shape=(3285263, 32, 4), dtype=float32
train_y: shape=(3285263,), dtype=int64
val_X: shape=(702434, 32, 4), dtype=float32
val_y: shape=(702434,), dtype=int64
test_X: shape=(702513, 32, 4), dtype=float32
test_y: shape=(702513,), dtype=int64


In [5]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes: int):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(64, 128)  # Adjust based on input size
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.global_pool(x)  # collapse spatial dims
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def save_model(self, filepath: str):
        torch.save(self.state_dict(), filepath)

    def load_model(self, filepath: str):
        self.load_state_dict(torch.load(filepath))
        self.eval()

class LSTMClassifier(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int, num_classes: int):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        return out

    def save_model(self, filepath: str):
        torch.save(self.state_dict(), filepath)

    def load_model(self, filepath: str):
        self.load_state_dict(torch.load(filepath))
        self.eval()

In [12]:
class TemporalCNN(nn.Module):
    """1D CNN with residual blocks for sequence windows shaped (B, T, F)."""
    def __init__(self, input_channels: int, num_classes: int, channels: tuple[int, ...] = (64, 128, 256)):
        super().__init__()
        layers = []
        in_ch = input_channels
        for out_ch in channels:
            layers.append(nn.Conv1d(in_ch, out_ch, kernel_size=3, padding=1))
            layers.append(nn.BatchNorm1d(out_ch))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(0.2))
            in_ch = out_ch
        self.backbone = nn.Sequential(*layers)
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(in_ch, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.transpose(1, 2)  # (B, F, T)
        x = self.backbone(x)
        return self.head(x)


class BiGRUClassifier(nn.Module):
    """Bidirectional GRU using final hidden state."""
    def __init__(self, input_size: int, hidden_size: int, num_layers: int, num_classes: int, dropout: float = 0.2):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        _, h_n = self.gru(x)
        h_last = torch.cat((h_n[-2], h_n[-1]), dim=1)  # concat directions
        return self.fc(h_last)


class TransformerClassifier(nn.Module):
    """Lightweight Transformer encoder pooling CLS token."""
    def __init__(
        self,
        input_size: int,
        num_classes: int,
        d_model: int = 128,
        nhead: int = 4,
        num_layers: int = 2,
        dim_feedforward: int = 256,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.input_proj = nn.Linear(input_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, num_classes),
        )
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.trunc_normal_(self.cls_token, std=0.02)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b = x.size(0)
        cls = self.cls_token.expand(b, -1, -1)
        x = torch.cat([cls, self.input_proj(x)], dim=1)
        enc = self.encoder(x)
        return self.head(enc[:, 0])

In [10]:
DATA_DIR = Path("C:/msc_2/DL/vitmma19-pw-bullflag/data")
PROCESSED_FILE = DATA_DIR / "processed_data.npz"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

def train_model(model, train_loader, criterion, optimizer, num_epochs, device):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

def validate_model(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    return val_loss / len(val_loader)


config = {
    'batch_size': 32,
    'learning_rate': 0.001,
    'num_epochs': 10
}

# Load dataset
data = np.load(DATA_DIR / "processed_data.npz")
train_data = data['train_X']
train_labels = data['train_y']

# Create DataLoader
# train_dataset = TensorDataset(
#     torch.tensor(train_data, dtype=torch.float32).unsqueeze(1),
#     torch.tensor(train_labels, dtype=torch.long),
# )
train_dataset = TensorDataset(
    torch.tensor(train_data, dtype=torch.float32),
    torch.tensor(train_labels, dtype=torch.long),
)
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)

# Initialize model, criterion, and optimizer
# model = CNNClassifier(num_classes=len(np.unique(train_labels))).to(device)
model = LSTMClassifier(input_size=train_data.shape[2], hidden_size=32, num_layers=1, num_classes=len(np.unique(train_labels))).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

# Train the model
train_model(model, train_loader, criterion, optimizer, config['num_epochs'], device)

# Save the model
torch.save(model.state_dict(), 'model_checkpoint.pth')

Using device: cuda
Epoch [1/10], Loss: 0.2218
Epoch [2/10], Loss: 0.2146
Epoch [3/10], Loss: 0.2129
Epoch [4/10], Loss: 0.2113
Epoch [5/10], Loss: 0.2102
Epoch [6/10], Loss: 0.2091


KeyboardInterrupt: 

CNN
Using device: cuda
Epoch [1/10], Loss: 0.2217
Epoch [2/10], Loss: 0.2147
Epoch [3/10], Loss: 0.2117
Epoch [4/10], Loss: 0.2102
Epoch [5/10], Loss: 0.2092
Epoch [6/10], Loss: 0.2089
Epoch [7/10], Loss: 0.2085
Epoch [8/10], Loss: 0.2083
Epoch [9/10], Loss: 0.2083
Epoch [10/10], Loss: 0.2082

In [29]:
train_dataset.tensors[0].shape

torch.Size([3285263, 1, 32, 4])

In [30]:
for inputs, labels in train_loader:
    print("Input batch shape:", inputs.shape)
    print("Label batch shape:", labels.shape)
    break


Input batch shape: torch.Size([32, 1, 32, 4])
Label batch shape: torch.Size([32])


In [13]:
MODEL_REGISTRY = {
    "temporal_cnn": lambda cfg, n_feats, n_classes: TemporalCNN(n_feats, n_classes),
    "bigru": lambda cfg, n_feats, n_classes: BiGRUClassifier(
        input_size=n_feats,
        hidden_size=cfg.get("hidden_size", 128),
        num_layers=cfg.get("num_layers", 2),
        num_classes=n_classes,
        dropout=cfg.get("dropout", 0.2),
    ),
    "transformer": lambda cfg, n_feats, n_classes: TransformerClassifier(
        input_size=n_feats,
        num_classes=n_classes,
        d_model=cfg.get("d_model", 128),
        nhead=cfg.get("nhead", 4),
        num_layers=cfg.get("num_layers", 2),
        dim_feedforward=cfg.get("dim_feedforward", 256),
        dropout=cfg.get("dropout", 0.1),
    ),
}

def load_processed_dataset(npz_file: Path) -> dict[str, np.ndarray]:
    if not npz_file.exists():
        raise FileNotFoundError(f"Missing file: {npz_file}")
    with np.load(npz_file) as data:
        return {key: data[key] for key in data.files}

def build_loader(x: np.ndarray, y: np.ndarray, batch_size: int, shuffle: bool) -> DataLoader:
    dataset = TensorDataset(
        torch.from_numpy(x).float(),
        torch.from_numpy(y).long(),
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def run_epoch(model, loader, criterion, optimizer=None, device="cpu"):
    training = optimizer is not None
    model.train(training)
    total_loss, total_correct, total_samples = 0.0, 0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        if training:
            optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        if training:
            loss.backward()
            optimizer.step()
        total_loss += loss.item() * xb.size(0)
        total_correct += (preds.argmax(1) == yb).sum().item()
        total_samples += xb.size(0)
    return total_loss / total_samples, total_correct / total_samples


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

arrays = load_processed_dataset(PROCESSED_FILE)
n_features = arrays["train_X"].shape[-1]
classes = np.unique(np.concatenate([arrays["train_y"], arrays.get("val_y", []), arrays.get("test_y", [])]))
num_classes = len(classes)

model_builder = MODEL_REGISTRY[config.get("model_name", "temporal_cnn")]
model = model_builder(config, n_features, num_classes).to(device)

train_loader = build_loader(arrays["train_X"], arrays["train_y"], config["batch_size"], shuffle=True)
val_loader = build_loader(arrays["val_X"], arrays["val_y"], config["batch_size"], shuffle=False) if "val_X" in arrays else None

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

best_val = float("inf")
for epoch in range(1, config["num_epochs"] + 1):
    train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer, device)
    if val_loader:
        val_loss, val_acc = run_epoch(model, val_loader, criterion, None, device)
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), "best_model.pt")
        print(f"Epoch {epoch}: train_loss={train_loss:.4f} train_acc={train_acc:.3f} "
                f"val_loss={val_loss:.4f} val_acc={val_acc:.3f}")
    else:
        torch.save(model.state_dict(), "best_model.pt")
        print(f"Epoch {epoch}: train_loss={train_loss:.4f} train_acc={train_acc:.3f}")


Epoch 1: train_loss=0.2357 train_acc=0.955 val_loss=0.5747 val_acc=0.883
Epoch 2: train_loss=0.2288 train_acc=0.955 val_loss=0.4579 val_acc=0.895
Epoch 3: train_loss=0.2260 train_acc=0.955 val_loss=0.4188 val_acc=0.942
Epoch 4: train_loss=0.2242 train_acc=0.955 val_loss=0.4586 val_acc=0.942
Epoch 5: train_loss=0.2227 train_acc=0.955 val_loss=0.4446 val_acc=0.888


KeyboardInterrupt: 

In [15]:
from pathlib import Path
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, TensorDataset

MODEL_REGISTRY = {
    "temporal_cnn": lambda cfg, n_feats, n_classes: TemporalCNN(n_feats, n_classes),
    "bigru": lambda cfg, n_feats, n_classes: BiGRUClassifier(
        input_size=n_feats,
        hidden_size=cfg.get("hidden_size", 128),
        num_layers=cfg.get("num_layers", 2),
        num_classes=n_classes,
        dropout=cfg.get("dropout", 0.2),
    ),
    "transformer": lambda cfg, n_feats, n_classes: TransformerClassifier(
        input_size=n_feats,
        num_classes=n_classes,
        d_model=cfg.get("d_model", 128),
        nhead=cfg.get("nhead", 4),
        num_layers=cfg.get("num_layers", 2),
        dim_feedforward=cfg.get("dim_feedforward", 256),
        dropout=cfg.get("dropout", 0.1),
    ),
}

def load_processed_dataset(npz_file: Path) -> dict[str, np.ndarray]:
    if not npz_file.exists():
        raise FileNotFoundError(f"Missing file: {npz_file}")
    with np.load(npz_file) as data:
        return {key: data[key] for key in data.files}

def build_loader(x: np.ndarray, y: np.ndarray, batch_size: int, shuffle: bool) -> DataLoader:
    dataset = TensorDataset(torch.from_numpy(x).float(), torch.from_numpy(y).long())
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def prepare_dataloaders(arrays: dict[str, np.ndarray], batch_size: int) -> dict[str, DataLoader]:
    loaders = {}
    for split in ("train", "val", "test"):
        x_key, y_key = f"{split}_X", f"{split}_y"
        if x_key in arrays and y_key in arrays and len(arrays[x_key]) > 0:
            loaders[split] = build_loader(arrays[x_key], arrays[y_key], batch_size, shuffle=(split == "train"))
    return loaders

def run_phase(model, loader, criterion, device, optimizer=None):
    if loader is None:
        return float("nan"), float("nan"), float("nan")
    training = optimizer is not None
    model.train(training)
    total_loss, total_correct, total_samples = 0.0, 0, 0
    preds_all, targets_all = [], []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        if training:
            optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        if training:
            loss.backward()
            optimizer.step()
        total_loss += loss.item() * xb.size(0)
        pred_labels = logits.argmax(1)
        total_correct += (pred_labels == yb).sum().item()
        total_samples += xb.size(0)
        preds_all.append(pred_labels.detach().cpu())
        targets_all.append(yb.detach().cpu())
    preds_concat = torch.cat(preds_all)
    targets_concat = torch.cat(targets_all)
    macro_f1 = f1_score(targets_concat, preds_concat, average="macro")
    return total_loss / total_samples, total_correct / total_samples, macro_f1

def train_and_validate():
    config = {
        'batch_size': 32,
        'learning_rate': 0.001,
        'num_epochs': 10
    }
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    arrays = load_processed_dataset(PROCESSED_FILE)
    n_features = arrays["train_X"].shape[-1]
    label_arrays = [arrays[key] for key in arrays if key.endswith("_y")]
    classes = np.unique(np.concatenate(label_arrays))
    num_classes = len(classes)

    loaders = prepare_dataloaders(arrays, config["batch_size"])

    model_builder = MODEL_REGISTRY[config.get("model_name", "temporal_cnn")]
    model = model_builder(config, n_features, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

    best_val_f1 = 0.0
    for epoch in range(1, config["num_epochs"] + 1):
        train_loss, train_acc, train_f1 = run_phase(model, loaders.get("train"), criterion, device, optimizer)
        val_loss, val_acc, val_f1 = run_phase(model, loaders.get("val"), criterion, device)

        msg = (f"Epoch {epoch:02d} | "
               f"train_loss={train_loss:.4f} train_acc={train_acc:.3f} train_f1={train_f1:.3f} | "
               f"val_loss={val_loss:.4f} val_acc={val_acc:.3f} val_f1={val_f1:.3f}")
        print(msg)

        if not np.isnan(val_f1) and val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pt")

    if "test" in loaders:
        test_loss, test_acc, test_f1 = run_phase(model, loaders["test"], criterion, device)
        print(f"Test  | loss={test_loss:.4f} acc={test_acc:.3f} f1={test_f1:.3f}")


train_and_validate()

Using device: cuda
Epoch 01 | train_loss=0.2355 train_acc=0.955 train_f1=0.140 | val_loss=0.6307 val_acc=0.883 val_f1=0.135
Epoch 02 | train_loss=0.2286 train_acc=0.955 train_f1=0.140 | val_loss=0.4519 val_acc=0.933 val_f1=0.139
Epoch 03 | train_loss=0.2262 train_acc=0.955 train_f1=0.141 | val_loss=0.5165 val_acc=0.925 val_f1=0.137
Epoch 04 | train_loss=0.2243 train_acc=0.955 train_f1=0.142 | val_loss=0.6051 val_acc=0.884 val_f1=0.135
Epoch 05 | train_loss=0.2227 train_acc=0.955 train_f1=0.142 | val_loss=0.4628 val_acc=0.898 val_f1=0.136
Epoch 06 | train_loss=0.2217 train_acc=0.955 train_f1=0.142 | val_loss=0.4065 val_acc=0.942 val_f1=0.139
Epoch 07 | train_loss=0.2208 train_acc=0.955 train_f1=0.143 | val_loss=0.4043 val_acc=0.942 val_f1=0.140
Epoch 08 | train_loss=0.2202 train_acc=0.955 train_f1=0.143 | val_loss=0.4947 val_acc=0.901 val_f1=0.137
Epoch 09 | train_loss=0.2196 train_acc=0.955 train_f1=0.143 | val_loss=0.4367 val_acc=0.941 val_f1=0.139
Epoch 10 | train_loss=0.2189 train_a

In [ ]:
# adat kimentés külön a teszteléshez? milyen eval kell egyáltalán?
# adatletöltés scriptbe 
# https://bmeedu-my.sharepoint.com/:u:/g/personal/gyires-toth_balint_vik_bme_hu/IQAlEFc87da4SLpRVTCs81KwATOAjf5GzI-IxEED_nGrjh0?e=eGgGec&download=1
# eval metrikák kódolása
# repo rendberakás, pathek, conténerezés